<a href="https://colab.research.google.com/github/DenisMaslyuk/-Udemy-FirstTask/blob/master/TravellingSalesman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import networkx as nx
import itertools

In [ ]:
N = 10 #Количество городов
k=100 #Популяция
iter=100 #Количество итераций

In [ ]:
#Генерация городов
cities = np.random.random_integers(0,2000,size=(N,N))
cities = np.maximum( cities, cities.transpose() )
for i in range(N):
  for j in range(N):
    if i<j:#Добавление расстояния на обратный путь
      cities[i,j]=cities[i,j]+np.random.random_integers(-200,200) 
    cities[i,i]=0
print(cities)

[[   0 1737  975 1768 1372 1130 1529 1651  960 1080]
 [1667    0  273 1795 1847 1789 1863  718 1847  971]
 [1115  385    0 1788 1824 1860  742 1884 1964 1050]
 [1590 1710 1982    0 1611  866 1717 1763 1725 1273]
 [1479 1794 1658 1786    0 1264  774 1474 1551 1582]
 [1029 1889 1924  760 1386    0 1802 2058 1718 1417]
 [1706 1803  810 1828  618 1618    0  412  673 1156]
 [1497  807 1877 1646 1391 1869  357    0 1622  302]
 [1079 1831 1871 1805 1430 1662  724 1604    0 1733]
 [1107  938 1155 1119 1606 1574 1078  404 1867    0]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated. Please call randint(0, 2000 + 1) instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: This function is deprecated. Please call randint(-200, 200 + 1) instead
  import sys


In [ ]:
def distance(cities,way):
  sum=0
  for i in range(len(way)-1):
      sum=sum+cities[way[i]][way[i+1]]
  return sum

Генерация пути при помощи множества

In [ ]:
def update_ar_way(ar_way):
  way=[i for i in range(N)]
  ar_way.clear()
  num = {i for i in range(N)}
  for i in range(k):
    for m in range(N):
      rn_num=np.random.choice(list(num))#Выбирается рандомно из множества,                                    
      num.discard(rn_num)               #тем самым исключая повторы
      way[m]=rn_num
    num = {i for i in range(N)}
    ar_way.append([list(way),distance(cities,way)])
  return ar_way

In [ ]:
def all_perms(elements):
    if len(elements) <=1:
        elements.append(0)
        yield elements
    else:
        for perm in all_perms(elements[1:]):
            for i in range(len(elements)):
                yield perm[:i] + elements[0:1] + perm[i:]

In [ ]:
print(list(itertools.permutations([1, 2, 3])))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]


In [ ]:
def brute_force(cities,N):
  ar_way=[]
  way=[i for i in range(1,N)]
  #all_ways=list(all_perms(way)) Перебор(N-1)!
  #Делаю полный перебор так как в некоторых случаех не явлется минимальным
  all_ways=list(itertools.permutations([i for i in range(N)])) 

  for i in range(len(all_ways)):
    ar_way.append([all_ways[i],distance(cities,all_ways[i])])
  ar_way.sort(key=lambda w: w[1])
  return ar_way



ГА
Берет путь и скрещивает со след двумя путями.
Так продалжается k//4 раз. В сумме получается N/2 путей.
Далее происходит замена первых N/2 путей новыми.
Срез пути происходит наполовину. Даллее при помощи множеств
ищутся повторения в первой части пути и если находит то рандомно заменяет на оставшиеся города.

In [ ]:
def gen_algoritm(cities,N,k,iter):
  ar_way=[] #Массив 
  sub_ar_way=[]
  ar_way=update_ar_way(ar_way)
  ar_way.sort(key=lambda w: w[1],reverse=True)

  for q in range(iter):
    sub_ar_way.clear()
    for i in range(k//4):
      temp_way=list(ar_way[i][0])#Буфер для срезов путей
      temp_way2=list(ar_way[i][0])
      cut_num=N//2
      temp_way[:cut_num]=ar_way[i+1][0][:cut_num]#Замена первой части пути на новую
      temp_way2[:cut_num]=ar_way[i+2][0][:cut_num]
      
      if len(set(temp_way))!=N: #Проверка на повторения
        for j in range(N//2):
          if temp_way[j] in temp_way[N//2:]:
            out_num={i for i in range(N)}
            out_num.difference_update(set(temp_way))
            temp_way[j]=np.random.choice(list(out_num))#Рандомный выбор из множества
      if len(set(temp_way2))!=N:
        for j in range(N//2):
          if temp_way2[j] in temp_way2[N//2:]:
            out_num={i for i in range(N)}
            out_num.difference_update(set(temp_way2))
            temp_way2[j]=np.random.choice(list(out_num))

      sub_ar_way.append([temp_way,distance(cities,temp_way)])
      sub_ar_way.append([temp_way2,distance(cities,temp_way2)])

    ar_way=update_ar_way(ar_way)
    temp_k=k//2
    ar_way[:temp_k]=sub_ar_way
    ar_way.sort(key=lambda w: w[1])

  return ar_way

In [ ]:
def draw_graph(ar_way):
  G = nx.Graph()
  current_way=ar_way[0][0]
  for i in range(N-1):
    G.add_edge(current_way[i], current_way[i+1])
  G.add_edge(current_way[-1],current_way[0])
  nx.draw(G, with_labels=True, font_weight='bold')

In [ ]:
%prun ar_way_bf=brute_force(cities,N)
print("БФ Минимальное расстояние =",ar_way_bf[0][1],"Путь:",ar_way_bf[0][0])
#draw_graph(ar_way_bf)

 БФ Минимальное расстояние = 6721 Путь: (2, 1, 7, 9, 3, 5, 0, 8, 6, 4)


In [ ]:
%prun ar_way_ga=gen_algoritm(cities,N,k,iter)
print("ГА Минимальное расстояние =",ar_way_ga[0][1],"Путь:",ar_way_ga[0][0])
#draw_graph(ar_way_ga)

 ГА Минимальное расстояние = 8772 Путь: [4, 8, 0, 5, 3, 1, 9, 7, 6, 2]
